In [1]:
# reloads modules when running again
%load_ext autoreload
%autoreload 2

In [17]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import scipy

from model_functions import *

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from darts import TimeSeries
from datetime import datetime
from sklearn.metrics.pairwise import paired_distances

import warnings
warnings.filterwarnings('ignore')

## Data loading

In [3]:
data, metadata = load_data()

### Constants

In [4]:
test_size = 10

## Data preprocessing

### Split to train-test

In [5]:
# sort data by collection date
data = data.sort_values(by="collection_date")

# split to train & test by test_size last samples
test_df = data.groupby('baboon_id').tail(test_size)
train_df = data.drop(test_df.index)
test_df = test_df.reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

# TODO: remove
# test_df = test_df[test_df["baboon_id"].isin(["Baboon_101", "Baboon_107", "Baboon_201"])]
# train_df = train_df[train_df["baboon_id"].isin(["Baboon_101", "Baboon_107", "Baboon_201"])]

In [6]:
meta_features_union = meta_features + ["month_sin", "month_cos"]
x_test, y_test = test_df[meta_features_union], test_df.drop(columns=meta_features_union)
x_test["interpolated"] = 0

### Interpolation

In [7]:
# aggregate to one sample per week
train_df = aggregate_samples(train_df.copy())

In [8]:
# TODO: delete i in knn_interpolation func
interpolated_df = knn_interpolation(train_df)
interpolated_df.to_csv("interpolated_df.csv")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


In [9]:
# interpolated_df = pd.read_csv("interpolated_df.csv")

In [10]:
interpolated_df["collection_date"] = pd.to_datetime(interpolated_df["collection_date"])
taxa_columns = [col for col in interpolated_df.columns if col not in meta_features_union + ["interpolated"]]

## Train

In [11]:
x_test.columns

Index(['sample', 'baboon_id', 'collection_date', 'age', 'sex', 'social_group',
       'month', 'rain_month_mm', 'diet_PC1', 'diet_PC2', 'diet_PC3',
       'month_sin', 'month_cos', 'interpolated'],
      dtype='object')

In [12]:
# TODO: delete all print
pred_df = predict(train_df, x_test)
pred_df.to_csv("pred_df.csv")

10 out of 800
20 out of 800
30 out of 800
40 out of 800
50 out of 800
60 out of 800
70 out of 800
80 out of 800
90 out of 800
100 out of 800
110 out of 800
120 out of 800
130 out of 800
140 out of 800
150 out of 800
160 out of 800
170 out of 800
180 out of 800
190 out of 800
200 out of 800
210 out of 800
220 out of 800
230 out of 800
240 out of 800
250 out of 800
260 out of 800
270 out of 800
280 out of 800
290 out of 800
300 out of 800
310 out of 800
320 out of 800
330 out of 800
340 out of 800
350 out of 800
360 out of 800
370 out of 800
380 out of 800
390 out of 800
400 out of 800
410 out of 800
420 out of 800
430 out of 800
440 out of 800
450 out of 800
460 out of 800
470 out of 800
480 out of 800
490 out of 800
500 out of 800
510 out of 800
520 out of 800
530 out of 800
540 out of 800
550 out of 800
560 out of 800
570 out of 800
580 out of 800
590 out of 800
600 out of 800
610 out of 800
620 out of 800
630 out of 800
640 out of 800
650 out of 800
660 out of 800
670 out of 800
680 

## Performance analysis

In [13]:
#pred_df = pd.read_csv("pred_df.csv")

In [14]:
pred_df

,index,sample,baboon_id,collection_date,age,sex,social_group,month,rain_month_mm,diet_PC1,...,g_Ruminococcaceae_UCG-011,g_Ruminococcaceae_UCG-013,g_Ruminococcaceae_UCG-014,g_Ruminococcus_1,g_Senegalimassilia,g_Slackia,g_Solobacterium,g_Streptococcus,g_Succinivibrio,g_Treponema_2
0,0,sample_12050-TCAGATACCAGC-407,Baboon_368,2003-08-28 00:00:00,7.633128,F,g_1.21,8,0.0,55.82963,...,0.000844,0.000657,0.005466,0.01001,0.003672,0.002238,0.017013,0.000172,0.003639,0.034166
1,1,sample_12050-CGTAGGTAGAGG-407,Baboon_368,2003-08-29 00:00:00,7.635866,F,g_1.21,8,0.0,55.82963,...,0.000843,0.000654,0.005443,0.010013,0.003676,0.002241,0.017039,0.000172,0.003624,0.034213
2,2,sample_12050-GGCCAGTTCCTA-407,Baboon_368,2003-09-12 00:00:00,7.674196,F,g_1.21,9,0.0,65.531243,...,0.00083,0.000491,0.004448,0.008973,0.004431,0.002704,0.020423,0.000184,0.003317,0.038917
3,3,sample_12049-TCTCATGTGGAG-407,Baboon_368,2003-09-22 00:00:00,7.701574,F,g_1.21,9,0.0,32.323606,...,0.001454,0.00291,0.011009,0.014028,0.001735,0.001022,0.00688,0.000241,0.002909,0.058808
4,4,sample_12049-ACCCTCAGCCCA-407,Baboon_368,2003-10-08 00:00:00,7.74538,F,g_1.21,10,0.0,30.589036,...,0.000743,0.00253,0.027419,0.01485,0.001682,0.001028,0.006871,0.000241,0.00234,0.035456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,795,sample_12052-GTTTCTTGTTGC-408,Baboon_343,2013-09-18 00:00:00,16.14511,M,g_1.22,9,0.0,40.64847,...,0.000427,0.002324,0.008043,0.016408,0.001187,0.001064,0.003963,0.001697,0.002579,0.006744
796,796,sample_12052-TTGGGAGCGAAG-408,Baboon_289,2013-09-18 00:00:00,10.26146,M,g_1.22,9,0.0,40.64847,...,0.000312,0.001791,0.005604,0.007069,0.001133,0.001,0.003083,0.001453,0.00113,0.011677
797,797,sample_12052-AATATACTCAAT-408,Baboon_289,2013-09-19 00:00:00,10.2642,M,g_1.22,9,0.0,36.2004,...,0.000307,0.002066,0.005425,0.007333,0.001086,0.001004,0.002956,0.001441,0.000986,0.004952
798,798,sample_12052-ATTTGTGGGTAG-408,Baboon_503,2013-09-19 00:00:00,16.61054,F,g_1.22,9,0.0,36.2004,...,0.000733,0.001746,0.006461,0.004875,0.001245,0.000968,0.003796,0.001484,0.000988,0.003677


In [24]:
d_matrix = pairwise_distances(pred_df[y_test.columns], y_test, metric="braycurtis").diagonal()
pd.DataFrame(d_matrix).to_csv("braycurtis_performance.csv")
braycurtis_score = d_matrix.mean()
print(braycurtis_score)

0.3787475079754418
